# InvokeAI in Google Colab

This is a tool to use Google colab to run the AI image generation tool: Invokeai (https://invoke-ai.github.io/InvokeAI/).
This automatically builds itself and can import your models, It can connect to Google drive to save your images.
It also has the option of running entirely from Google drive to vastly speed up the time it takes to start, However, over 20Gb of Google Drive space is required! However it changes subsequent startup times from 15-20 mins to 4-5 mins.

Make sure to enable GPU, Edit > Notebook Settings > Hardware accelerator > GPU

To start, Click "Runtime" > "Run All". Alternaivly you can click the "play" button on each step below one after the other, No need to wait for the previous steps to finish as they will join a queue.<br>

###Note: Currently, the Gui is bugged for V3.0.0 | To fix this, you need to use a "Servio" or "Localtunnel" connection type (NGROK does NOT work for this workaround) After launching the page, enable "Insecure content" in "Site setings" - A back-end fix is being worked on. - https://github.com/invoke-ai/InvokeAI/issues/3693


In [ ]:
#@title 1. Configuration { display-mode: "form" }
#@markdown #Instance Type
#@markdown **Google_Drive** = Stores ALL files in your google drive, This will take over 20Gb of space (Per Version), but will start up much faster as it does not download invokeai at each run. To make it download a fresh version simply delete the folder "/content/drive/MyDrive/invokeai/invokeaiVERSION/noUpdate, or indeed the whole "invokeaiVERSION" folder as outputs are stored elsewhere.<br>
#@markdown **Persistent** = Stores only your images in Google drive, in a folder called "/invokeai/invokeaiOutput" with the invoke version number on the end.<br>
#@markdown **Temporary** (NOT recomended) = Everything is stored in the runtime and is removed when the runtime ends or crashes, make sure to download your images! <br><br>
#@markdown **Rough Startup times:** <br>
#@markdown V2.3.5 = 10-12 mins <br>
#@markdown V3 (Temporary or Persistant modes or Google_Drive First run.) = 15-20 mins (Full) | 10 mins (Custom model ONLY) <br>
#@markdown V3 "Google_Drive" = Subsiquent runs, 4-5 mins, or 6-7 mins if it is updating.
Version = "Version 3" #@param ['Version 3','2.3.5.post2'] {type:"string"}
Type = "Google_Drive" #@param ['Google_Drive','Persistent','Temporary'] {type:"string"}

#@markdown ---

#@markdown #Connection Type.
#@markdown Serveo: Mediumly stable, requires no setup. <br>
#@markdown Localtunnel: Less stable, also requires no setup. It is an alternate if for some reson 'Serveo' is not working. It also requires an extra step at connection time.<br>
#@markdown NGROK: Highly stable but needs a little setting up, An NGROK token is required, sign up for free and get one here: https://dashboard.ngrok.com/get-started/your-authtoken Once you have the token, please put it in below.<br>
connection_type = "Serveo"  #@param ["Serveo","Localtunnel","NGROK"]
ngrok_token = "None" #@param ['None'] {allow-input: true}

#@markdown ---

#@markdown #Custom Models
#@markdown Skip Default Models: <br>
#@markdown When installing for the first time, you can skip the default models to save about 10 mins and 30GB. This setting has no effect on a re-run (it does not add or remove existing models). You MUST specify a custom model if you choose to skip the default models. <br>
#@markdown Skipping default models only works with V3+ <br>
skip_default_models = "No" #@param ["Yes","No"]
#@markdown Custom Diffuser Model
custom_diffuser_hugging_face_id = "None" #@param ["None","kebab111/crosskemono", "dreamlike-art/dreamlike-photoreal-2.0", "stablediffusionapi/disney-pixal-cartoon"] {allow-input: true}
#@markdown Custom Safetensor or Checkpoint models <br>
custom_model_url = "None" #@param ["None","https://huggingface.co/ffggggfg/Crosskemono/resolve/main/crosskemonoFurryModel_crosskemono25.safetensors"] {allow-input: true}
custom_vae_url = "None" #@param ["None","https://huggingface.co/ffggggfg/Crosskemono/resolve/main/Crosskemono2.vae.pt"] {allow-input: true}

#@markdown If you have selected a custom diffuser model it will be auto imported and it is selectable from the dropdown menu in app. <br>
#@markdown Custom .safetensor or .checkpoint models are stored in /content/invokeai/CustomModels and must be manually added via the GUI after it starts. Your Custom VAE will be stored in /content/invokeai/CustomVae <br>

#@markdown ---

#@markdown #NSFW Checker
#@markdown This checks outputs for "Not Safe For Work" content and obscures the image. This is not 100% reliable.
nsfw_checker = "Enabled" #@param ['Enabled' ,'Disabled'] {type:"string"}

#@markdown ---

#@markdown #Xformers
#@markdown Xformers makes this "memory efficient" so larger images (above 1024x1024) can be made, However it can be less stable and may generate blank outputs. It is recomended that you set it to "Enabled" unless you experiance issues with your models.
xformers = "Enabled" #@param ['Enabled' ,'Disabled'] {type:"string"}


In [ ]:
#@title 2.a. Build and Configure. (V2.3.5.post2) { display-mode: "form" }
#@markdown This takes about 10-12 mins to finish + some extra time to download custom Checkpoint / Safetensor models.<br>
#@markdown Version 2.3.5.post2 Downloads Checkpoint models now, and they must be manually added via the GUI.<br>
#@markdown It will download and auto-add Diffuser models during startup.

from IPython.display import clear_output
if Version == "2.3.5.post2":
  import os

  #Set up temporary storage if running in "Temporary" mode.
  if Type == "Temporary":
      file_path = '/content/invokeai'
      outpath = '/content/invokeai/output'
      noUpdate = '/content/invokeai/noUpdate'

      if not os.path.exists(file_path):
        os.makedirs(file_path)

  # Mount and set up Google drive if running in "Persistent" mode.
  if Type == "Persistent":
    file_path = '/content/invokeai'
    outpath = '/content/drive/MyDrive/invokeai/invokeaiOutput2.3.5.post2'
    noUpdate = '/content/invokeai/noUpdate'

    from google.colab import drive
    if not os.path.exists(file_path):
      drive.mount('/content/drive')
      if not os.path.exists(file_path):
        os.makedirs(file_path)

  # Mount and set up Google drive if running in "Google_Drive" mode.
  if Type == "Google_Drive":
    file_path = '/content/drive/MyDrive/invokeai/invokeai2.3.5.post2'
    outpath = '/content/drive/MyDrive/invokeai/invokeaiOutput2.3.5.post2'
    noUpdate = '/content/drive/MyDrive/invokeai/invokeai2.3.5.post2/noUpdate'

    from google.colab import drive
    if not os.path.exists(file_path):
      drive.mount('/content/drive')
      if not os.path.exists(file_path):
        os.makedirs(file_path)

  #Install Python + Pip
  !sudo apt-get update -y
  !sudo apt-get install python3.9 python3-pip python3.9-distutils python3.9-dev libpython3.9-dev python3.9-venv -y
  #!sudo apt-get install python3.10 python3-pip python3.10-distutils python3.10-dev libpython3.10-dev python3.10-venv -y

  #Change alternatives
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 3

  #Update pip
  %cd {file_path}
  !curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
  !python3 get-pip.py --force-reinstall
  !python -m pip install --upgrade pip
  !pip install --upgrade setuptools
  !pip install pillow==9.3.0

  #Handle colab dependencies
  #!ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.9/dist-packages/google
  #!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

  #Handle colab dependencies
  !ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.9/dist-packages/google
  !python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

  #Create InvokeAI root
  import os
  os.environ['INVOKEAI_ROOT'] = file_path
  if not os.path.exists(file_path):
    os.makedirs(invokeai_root)

  #On 2nd run, do an "upgrade" to get system variables to load quickly.
  if os.path.exists(noUpdate):
    !python -m venv .venv --prompt InvokeAI
    !source .venv/bin/activate; pip install InvokeAI[xformers]==2.3.5.post2 --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117 --upgrade

  #Create the virtual environment + Download default models
  %cd {file_path}
  !python -m venv .venv --prompt InvokeAI
  if not os.path.exists(noUpdate):
    !source .venv/bin/activate; python -m pip install --upgrade pip
    !source .venv/bin/activate; pip install InvokeAI[xformers]==2.3.5.post2 --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117
    !source .venv/bin/activate; python -m pip uninstall torchmetrics==1.0.0 -y
    !source .venv/bin/activate; python -m pip install torchmetrics==0.7.0
    !mkdir {outpath}
    !cd {file_path}
    !source .venv/bin/activate; invokeai-configure --yes
    !mkdir {noUpdate}

  #Edit invokeai.init
  !sed -i 's/--max_loaded_models=2/--max_loaded_models=1/' invokeai.init
  if xformers == "Disabled":
    !sed -i 's/--xformers/--no-xformers/' invokeai.init
  if xformers == "Enabled":
    !sed -i 's/--no-xformers/--xformers/' invokeai.init
  if nsfw_checker == "Disabled":
    !sed -i 's/--nsfw_checker/--no-nsfw_checker/' invokeai.init
  if nsfw_checker == "Enabled":
    !sed -i 's/--no-nsfw_checker/--nsfw_checker/' invokeai.init

  #Download custom checkpoint models
  if custom_model_url != "None":
    !mkdir {file_path}/CustomModels
    %cd {file_path}/CustomModels
    !wget -nc {file_path}/CustomModels {custom_model_url}

  #Download custom VAE
  if custom_vae_url != "None":
    !mkdir {file_path}/CustomVae
    %cd {file_path}/CustomVae
    !wget -nc {file_path}/CustomVae {custom_vae_url}

  #Clear Output
  clear_output()

In [ ]:
#@title 2.b. Build and Configure. (v3.0.1rc2) { display-mode: "form" }
#@markdown This takes about 15 mins to finish + some extra time to download custom models. <br>
#@markdown Version 3 Downloads all models during the build phase.<br>
#@markdown It will auto add all models to the app.

from IPython.display import clear_output
if Version == "Version 3":

  #Set up temporary storage if running in "Temporary" mode.
  if Type == "Temporary":
      file_path = '/content/invokeai'
      outpath = '/content/invokeai/output'
      noUpdate = '/content/invokeai/noUpdate'
      import os
      if not os.path.exists(file_path):
        os.makedirs(file_path)

  # Mount and set up Google drive if running in "Persistent" mode.
  if Type == "Persistent":
    file_path = '/content/invokeai'
    outpath = '/content/drive/MyDrive/invokeaiOutputV3'
    noUpdate = '/content/invokeai/noUpdate'
    import os
    from google.colab import drive
    if not os.path.exists(file_path):
      drive.mount('/content/drive')
      if not os.path.exists(file_path):
        os.makedirs(file_path)

  # Mount and set up Google drive if running in "Google_Drive" mode.
  if Type == "Google_Drive":
    file_path = '/content/drive/MyDrive/invokeai/invokeaiV3'
    outpath = '/content/drive/MyDrive/invokeai/invokeaiOutputV3'
    noUpdate = '/content/drive/MyDrive/invokeai/invokeaiV3/noUpdate'

    import os
    from google.colab import drive
    if not os.path.exists(file_path):
      drive.mount('/content/drive')
      if not os.path.exists(file_path):
        os.makedirs(file_path)

  #Install Python + Pip
  !sudo apt-get update -y
  !sudo apt-get install python3.9 python3-pip python3.9-distutils python3.9-dev libpython3.9-dev python3.9-venv -y

  #When updating to Python V3.10 File permissions on the Installed files are wrong. They can be seen in this File Viewer but can NOT be seen or used by the app itself.
  #!sudo apt-get install python3.10 python3-pip python3.10-distutils python3.10-dev libpython3.10-dev python3.10-venv -y

  #Change alternatives
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 3

  #Update pip
  %cd {file_path}
  !curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
  !python3 get-pip.py --force-reinstall
  !python -m pip install --upgrade pip
  !pip install --upgrade setuptools
  !pip install pillow==9.3.0


  #Handle colab dependencies - Archive this code for the moment.
  #!ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.9/dist-packages/google
  #!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor


  #Create InvokeAI root
  import os
  os.environ['INVOKEAI_ROOT'] = file_path
  if not os.path.exists(file_path):
    os.makedirs(invokeai_root)


  #Create the virtual environment + Download default Models
  %cd {file_path}

  #On 2nd run, Do an "upgrade" to get system variables to load quickly.
  if os.path.exists(noUpdate):
    !python -m venv .venv --prompt InvokeAI

    #Update via a GIT pull
    #!source .venv/bin/activate; pip install "invokeai @ https://github.com/invoke-ai/InvokeAI/archive/refs/tags/v3.0.1rc2.zip" --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117 --upgrade

    #Update fully with PIP
    !source .venv/bin/activate; pip install InvokeAI[xformers]==3.0.1rc2 --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117 --upgrade

  if not os.path.exists(noUpdate):
    !python -m venv .venv --prompt InvokeAI
    !source .venv/bin/activate; python -m pip install --upgrade pip

    #Install via a GIT pull
    #!source .venv/bin/activate; pip install "invokeai @ https://github.com/invoke-ai/InvokeAI/archive/refs/tags/v3.0.1rc2.zip" --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117

    #Install flully with PIP.
    !source .venv/bin/activate; pip install InvokeAI[xformers]==3.0.1rc2 --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117

    !mkdir {outpath}
    !cd {file_path}

    if skip_default_models == "Yes":
      !source .venv/bin/activate; invokeai-configure --yes --skip-sd-weights
    if skip_default_models == "No":
      !source .venv/bin/activate; invokeai-configure --yes

    !mkdir {noUpdate}


  ## Edit invokeai.yaml
  !sed -i 's/max_loaded_models: 3/max_loaded_models: 2/' invokeai.yaml
  !sed -i 's/max_cache_size: 6.0/max_cache_size: 6.0/' invokeai.yaml
  !sed -i 's/max_vram_cache_size: 2.75/max_vram_cache_size: 4.0/' invokeai.yaml
  if xformers == "Disabled":
    !sed -i 's/xformers_enabled: true/xformers_enabled: false/' invokeai.yaml
  if xformers == "Enabled":
    !sed -i 's/xformers_enabled: false/xformers_enabled: true/' invokeai.yaml
  if nsfw_checker == "Disabled":
    !sed -i 's/nsfw_checker: true/nsfw_checker: false/' invokeai.yaml
  if nsfw_checker == "Enabled":
    !sed -i 's/nsfw_checker: false/nsfw_checker: true/' invokeai.yaml

  #Download custom checkpoint models
  if custom_model_url != "None":
    %cd {file_path}/autoimport/main
    !wget -nc {file_path}/autoimport/main {custom_model_url}

  #Download custom VAE
  if custom_vae_url != "None":
    %cd {file_path}/autoimport/main
    !wget -nc {file_path}/autoimport/main {custom_vae_url}

  #Import Diffuser Model
  if custom_diffuser_hugging_face_id != "None":
    %cd {file_path}/autoimport/main
    !git clone --depth 1 https://huggingface.co/{custom_diffuser_hugging_face_id}

  %cd {file_path}

  #Clear Output
  clear_output()

In [ ]:
#@title 3. Start InvokeAI. { display-mode: "form" }
#@markdown ### Starting the App
#@markdown This step takes either 2-3 mins to start InvokeAI or 5-6 mins if you have selected a diffuser model. <br>
#@markdown Diffuser models that are auto imported are NOT selected by default and must be chosen from the menu in app.

#@markdown ### Notes about connection types.
#@markdown #### Serveo = Almost, always connects however, it can drop your connection if a HTML error occurs, Simply wait for the images to finish generating, stop an re-start this step.
#@markdown #### Localtunnel = Once it gets going it is quite stable but often has "502" Errors, You must wait for THEM to fix it, please try another connection type.
#@markdown #### NGROK = Very stable but requires a token, see the "configuration" step for more details.

#@markdown ---

%cd {file_path}
import os

if Version == "2.3.5.post2":
  if connection_type == "Serveo":
    if custom_diffuser_hugging_face_id != "None":
      !ssh -o StrictHostKeyChecking=no -o ServerAliveInterval=60 -R 80:localhost:9090 serveo.net & !source {file_path}/.venv/bin/activate; invokeai --web --host 0.0.0.0 --outdir={outpath} --autoimport {custom_diffuser_hugging_face_id}
    if custom_diffuser_hugging_face_id == "None":
      !ssh -o StrictHostKeyChecking=no -o ServerAliveInterval=60 -R 80:localhost:9090 serveo.net & !source {file_path}/.venv/bin/activate; invokeai --web --host 0.0.0.0 --outdir={outpath}

  if connection_type == "Localtunnel":
    print("How to connect to localtunnel:");
    print("A localtunnel Interface connection is generated here, To use this, please do the following ")
    print("1. Copy this IP address")
    !curl ipv4.icanhazip.com
    print("2. Click the random 'https://XXX-YYY-ZZZ.loca.lt' link that is generated below.")
    print("3. Paste the IP into the provided box and submit. ")
    print(" ")
    print("Note: An error of '502 Bad Gateway' typically is an error at Localtunnels end. A '504 Gateway Time-out' Error means invokeai has not started yet.")
    print(" ")
    !npm install -g localtunnel
    if custom_diffuser_hugging_face_id != "None":
      !npx localtunnel --port 9090 & !source {file_path}/.venv/bin/activate; invokeai --web --host 0.0.0.0 --outdir={outpath} --autoimport {custom_diffuser_hugging_face_id}
    if custom_diffuser_hugging_face_id == "None":
      !npx localtunnel --port 9090 & !source {file_path}/.venv/bin/activate; invokeai --web --host 0.0.0.0 --outdir={outpath}


  if connection_type == "NGROK":
    if ngrok_token == "None":
      print("You have Selected NGROK but did not supply an NGROK token.")
      print("Falling back to a 'Serveo' connection type.")
      print("Please either add an NGROK token to step 1, re-run step 1, then re-run this step, or just re-run this step to use 'Servio'.")
      connection_type = "Serveo"
    if ngrok_token != "None":
      !pip install pyngrok --quiet
      from pyngrok import ngrok
      ngrok.kill()
      ngrok.set_auth_token(ngrok_token)
      public_url = ngrok.connect(9090).public_url
      print(f'InvokeAI Public URL: {public_url}')
      if custom_diffuser_hugging_face_id != "None":
        !source {file_path}/.venv/bin/activate; invokeai --web --host 0.0.0.0 --outdir={outpath} --autoimport {custom_diffuser_hugging_face_id}
      if custom_diffuser_hugging_face_id == "None":
        !source {file_path}/.venv/bin/activate; invokeai --web --host 0.0.0.0 --outdir={outpath}


if Version == "Version 3":

  if connection_type == "Serveo":
    !ssh -o StrictHostKeyChecking=no -o ServerAliveInterval=60 -R 80:localhost:9090 serveo.net & !source {file_path}/.venv/bin/activate; invokeai-web --outdir={outpath}

  if connection_type == "Localtunnel":
    print("How to connect to localtunnel:");
    print("A localtunnel Interface connection is generated here, To use this, please do the following ")
    print("1. Copy this IP address")
    !curl ipv4.icanhazip.com
    print("2. Click the random 'https://XXX-YYY-ZZZ.loca.lt' link that is generated below.")
    print("3. Paste the IP into the provided box and submit. ")
    print(" ")
    print("Note: An error of '502 Bad Gateway' typically is an error at Localtunnels end. A '504 Gateway Time-out' Error means invokeai has not started yet.")
    print(" ")
    !npm install -g localtunnel
    !npx localtunnel --port 9090 & !source {file_path}/.venv/bin/activate; invokeai-web --outdir={outpath}

  if connection_type == "NGROK":
    if ngrok_token == "None":
      print("You have Selected NGROK but did not supply an NGROK token.")
      print("Falling back to a 'Serveo' connection type.")
      print("Please either add an NGROK token to step 1, re-run step 1, then re-run this step, or just re-run this step to use 'Servio'.")
      connection_type = "Serveo"
    if ngrok_token != "None":
      !pip install pyngrok --quiet
      from pyngrok import ngrok
      ngrok.kill()
      ngrok.set_auth_token(ngrok_token)
      public_url = ngrok.connect(9090).public_url
      print(f'InvokeAI Public URL: {public_url}')
      !source {file_path}/.venv/bin/activate; invokeai-web --outdir={outpath}